# ***Modelo de clasificación de casos alarmantes y no alarmantes de Dengue en el municipio de Casanare***

Para analizar la situación del dengue en el municipio de Casanare, se ha seleccionado un dataset que recoge los casos reportados desde el 23 de octubre de 2022. En este dataset, los casos están clasificados entre aquellos con síntomas alarmantes y aquellos sin riesgo inmediato. Esta información nos permite tener una visión más clara del impacto y la gravedad de los casos en la región. Cabe destacar que la última actualización de este dataset fue el 17 de febrero de 2023, lo que garantiza que estamos trabajando con datos recientes y relevantes para nuestra evaluación. (Fuente: Departamento de Epidemiología, ESE Salud Yopal, 2023).

***Propuesta***

Como propuesta para el concurso de Datos a la U, vamos a desarrollar un modelo de *Machine learning* que permita identificar de manera eficaz los casos que requieren atención inmediata. Esta herramienta no solo busca apoyar a los profesionales de la salud en la toma de decisiones, sino que también contribuirá a una respuesta más ágil y coordinada frente a brotes de dengue. Al facilitar una intervención temprana, podemos mejorar considerablemente la gestión de la salud pública y salvar vidas.


## ***Importaciones***

In [77]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import os
import sys
import anthropic
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.colors import black, gray,white
from datetime import date

## ***Estudio del Dataset***

In [ ]:
sep = os.path.sep
dir_actual = os.path.abspath('')
PATH = sep.join(dir_actual.split(sep)[:-1])
DIR_DATA = PATH + '{0}Dengue{0}Data{0}'.format(os.sep)
sys.path.append(PATH) if PATH not in list(sys.path) else None

# Crear el directorio si no existe
if not os.path.exists(DIR_DATA):
    os.makedirs(DIR_DATA)  
    print(f"Carpeta creada en la ruta: {DIR_DATA}")
else:
    print("La carpeta ya existe:", DIR_DATA)
    
DIR_DATA

In [6]:
filename = DIR_DATA + 'Casos_de_Dengue.csv'
data = pd.read_csv(filename, sep = ',')

***Procedemos a observar el dataset***

In [ ]:
data

In [ ]:
data.info()

Observamos que el dataset ostenta 9 variables de tipo numerico y 45 variables de tipo objeto. El Dataset no alberga valores nulos.

In [ ]:
data.columns[data.isnull().any()]

In [ ]:
categorical_cols = data.select_dtypes(include=['object']).columns.tolist() #Guardamos en "categorical_ cols" las variables categoricas.


print("Categorical Columns:")
for col in categorical_cols:
    print(col)

Removemos del dataset las variables categoricas que determinamos no aportan mucho al analisis. Esto lo hacemos para mejorar el rendimiento y la interpretabilidad del modelo.

In [11]:
removed_categories = ['orden', 'semana', 'año', 'cod_pre', 'cod_sub', 'cod_pais_o', 
                      'cod_dpto_o', 'cod_mun_o', 'fec_not', 'ini_sin_', 'fec_con_', 
                      'nombre_nacionalidad', 'ndep_resi', 'nmun_resi', 'fuente_', 
                      'conducta', 'nom_eve', 'nom_upgd', 'fecha_nto_', 
                      'fec_arc_xl', 'fec_aju_', 'desplazami'] 

for variable in removed_categories:
    if variable in categorical_cols: 
        categorical_cols.remove(variable)

In [12]:
data = data.drop(columns = removed_categories)

In [ ]:
categorical_cols

Convertimos los datos categoricos en representaciones numericas, esto con el objetivo de facilitar el entrenamiento del modelo. Para ello usaremos una funcion de "pandas": "factorize". Sobreescribiendo ***data[categorie]*** con los valores numéricos.

In [14]:
for categorie in categorical_cols:
    data[categorie], tld_enum = pd.factorize(data[categorie]) #Convertimos los valores categoricos en valores numericos enteros. 

In [ ]:
print("\nCategorías originales y su asignación numérica:")
for i, category in enumerate(tld_enum):
    print(f"{category} -> {i}")

Asi quedaria nuestro el Dataset al extraer las variables fundamentales para el entrenamiento del modelo

In [ ]:
data

## ***Entrenamiento del modelo***

***Nuestro conjunto de datos se dividirá de la siguiente manera***:

- ***70% para el entrenamiento***: Este segmento será utilizado para enseñar al modelo a reconocer patrones en los casos de Dengue con sintomas alarmantes y no alarmantes
- ***20% para pruebas (testing)***: Esta parte se reservará para evaluar el rendimiento del modelo, verificando su capacidad para realizar predicciones con casos nuevos.
- ***10% para validación***: Este subconjunto se usará durante el proceso de ajuste de hiperparámetros y validación cruzada, asegurando que el modelo generalice correctamente y evite el sobreajuste.




In [17]:
data_validation, data_temp = train_test_split(data, test_size=0.9, random_state=42)

In [18]:
train_size = 0.7 / 0.9  # Ajustar la proporción relativa dentro del 90%
data_train, data_test = train_test_split(data_temp, test_size=1-train_size, random_state=42)

In [ ]:
train_filename = DIR_DATA + 'Casos_de_Dengue_train.csv'
test_filename = DIR_DATA + 'Casos_de_Dengue_test.csv'
validation_filename = DIR_DATA + 'Casos_de_Dengue_validation.csv'

data_train.to_csv(train_filename, index=False)
data_test.to_csv(test_filename, index=False)
data_validation.to_csv(validation_filename, index=False)

print(f"Datos de entrenamiento guardados en: {train_filename}")
print(f"Datos de prueba guardados en: {test_filename}")
print(f"Datos de validación guardados en: {validation_filename}")

In [ ]:
#Comprobams la distribución de los dataset
print(f"Entrenamiento: {len(data_train)} ({len(data_train) / len(data) * 100:.2f}%)")
print(f"Prueba: {len(data_test)} ({len(data_test) / len(data) * 100:.2f}%)")
print(f"Validación: {len(data_validation)} ({len(data_validation) / len(data) * 100:.2f}%)")

In [21]:
x_train = data_train.drop(columns=['clasfinal'])  
y_train = data_train['clasfinal']

In [22]:
x_test = data_test.drop(columns=['clasfinal'])  
y_test = data_test['clasfinal']

In [23]:
x_validation = data_validation.drop(columns=['clasfinal'])  
y_validation = data_validation['clasfinal']

In [24]:
#Normalizamos los datos
scaler = StandardScaler()
x_train_normalized = scaler.fit_transform(x_train)
x_test_normalized = scaler.transform(x_test)
x_validation_normalized = scaler.transform(x_validation)

***Escogimos tres modelos de machine learning para poder probar cual viene mejor para este caso: RandomForest, SGDClassifier y SVC***

In [ ]:

rf_model = RandomForestClassifier(n_estimators=150,max_depth=10,random_state=42)
rf_model.fit(x_train_normalized, y_train)

In [ ]:
sgd_model = SGDClassifier(loss='log_loss', penalty='elasticnet', alpha=0.01, max_iter=1000)
sgd_model.fit(x_train_normalized, y_train)

In [ ]:
svm_model = SVC(kernel='poly', C=1.0, random_state=42) 
svm_model.fit(x_train_normalized, y_train)

## ***Cross validation***


Para comprobar si los modelo son capaces de generalizar datos nuevos, los someteremos a una validación cruzada.

In [28]:
cv_folds = 5

In [ ]:
rf_scores = cross_val_score(rf_model, x_train_normalized, y_train, cv=cv_folds)
print("Cross Validation: RandomForest")
print(rf_scores.mean())

In [ ]:
sdg_scores = cross_val_score(sgd_model, x_train_normalized, y_train, cv=cv_folds)
print("Cross Validation: SGD")
print(sdg_scores.mean())

In [ ]:
svm_scores = cross_val_score(svm_model, x_train_normalized, y_train, cv=cv_folds)
print("Cross Validation: SVM")
print(svm_scores.mean())

Ahora, probaremos cada modelo con el dataset de prueba, y el escogido sera el que obtenga mejores resultados en las metricas medidas.

***RandomForest***

In [32]:
y_predRF = rf_model.predict(x_test_normalized)
RF_model_accuracy = accuracy_score(y_test,y_predRF)
RF_precision = precision_score(y_test,y_predRF)
RF_recall = recall_score(y_test,y_predRF)
RF_f1 = f1_score(y_test,y_predRF)

***Descenso Estocástico del Gradiente (SGD)***

In [33]:
y_predSDG = sgd_model.predict(x_test_normalized)
SDG_model_accuracy = accuracy_score(y_test,y_predSDG)
SDG_precision = precision_score(y_test,y_predSDG)
SDG_recall = recall_score(y_test,y_predSDG)
SDG_f1 = f1_score(y_test,y_predSDG)


***Máquinas de Soporte Vectorial (SVM)***

In [34]:
y_predSVM = svm_model.predict(x_test_normalized)
SVM_model_accuracy = accuracy_score(y_test,y_predRF)
SVM_precision = precision_score(y_test,y_predRF)
SVM_recall = recall_score(y_test,y_predRF)
SVM_f1 = f1_score(y_test,y_predRF)

## ***Desempeño de cada modelo***

In [ ]:
print("Data Evaluation: RandomForest")
print(f"Accuracy: {RF_model_accuracy}")
print(f"Precision: {RF_precision}")
print(f"Recall: {RF_recall}")
print(f"F1 Score: {RF_f1}") 

In [ ]:
print("Data Evaluation: SGD")
print(f"Accuracy: {SDG_model_accuracy}")
print(f"Precision: {SDG_precision}")
print(f"Recall: {SDG_recall}")
print(f"F1 Score: {SDG_f1}")

In [ ]:
print("Data Evaluation: SVM")
print(f"Accuracy: {SVM_model_accuracy}")
print(f"Precision: {SVM_precision}")
print(f"Recall: {SVM_recall}")
print(f"F1 Score: {SVM_f1}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Resultados de ejemplo (sustituye por los valores reales)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
random_forest = [RF_model_accuracy, RF_precision, RF_recall, RF_f1]
sgd = [SDG_model_accuracy, SDG_precision, SDG_recall, SDG_f1]
svm = [SVM_model_accuracy, SVM_precision, SVM_recall, SVM_f1]

# Configuración del gráfico
x = np.arange(len(metrics))  # Posiciones de las métricas
width = 0.25  # Ancho de las barras

# Creación de la figura
fig, ax = plt.subplots(figsize=(12, 7))

# Barras para cada modelo
bars_rf = ax.bar(x - width, random_forest, width, label='Random Forest', color='skyblue')
bars_sgd = ax.bar(x, sgd, width, label='SGD', color='salmon')
bars_svm = ax.bar(x + width, svm, width, label='SVM', color='lightgreen')

# Añadir valores a las barras
for bars in [bars_rf, bars_sgd, bars_svm]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',  # Mostrar valor con 2 decimales
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # Desplazamiento vertical
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10, color='black')

# Etiquetas y detalles del gráfico
ax.set_xlabel('Métricas', fontsize=12)
ax.set_ylabel('Valores', fontsize=12)
ax.set_title('Comparación de Modelos con Métricas', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(metrics, fontsize=11)
ax.legend()

# Mostrar el gráfico
plt.tight_layout()
plt.show()



## ***Matrices de confusion***

In [39]:

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_test, y_pred, title):
    cm = confusion_matrix(y_test, y_pred)
    
    # Calcular los porcentajes
    total = cm.sum()
    cm_percentage = cm / total * 100

    # Crear la gráfica
    plt.figure(figsize=(5, 5))
    sns.heatmap(cm_percentage, annot=True, fmt=".2f", cmap="Blues", cbar=False)
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    plt.title(title)
    plt.show()

    # Mostrar los valores de TP, TN, FP, FN
    TP = cm[1, 1]
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    
    print(f"Verdaderos Positivos (TP): {TP} ({(TP/total)*100:.2f}%)")
    print(f"Verdaderos Negativos (TN): {TN} ({(TN/total)*100:.2f}%)")
    print(f"Falsos Positivos (FP): {FP} ({(FP/total)*100:.2f}%)")
    print(f"Falsos Negativos (FN): {FN} ({(FN/total)*100:.2f}%)")


In [ ]:
plot_confusion_matrix(y_predRF, y_test, 'Matriz de Confusión - random forest')

In [ ]:
plot_confusion_matrix(y_predSDG, y_test, 'Matriz de Confusión - SDG')

In [ ]:
plot_confusion_matrix(y_predSVM, y_test, 'Matriz de Confusión - SVM')

Analizando los resultados obtenidos(las validaciones cruzadas de cada modelo, las metricas y las matrices de confusión realizadas), podemos decir que el modelo de machine learning que usaremos para poder clasificar los casos de Dengue en alarmantes y no alarmantes, sera el ***RandomForest***. Es el modelo que más parece captar la naturaleza del dataset, y más se ajusta a lo esperado. 

# ***Validación del modelo***

In [43]:
y_pred_validation_rf = rf_model.predict(x_validation_normalized)

In [ ]:
data_validation['pred_clasfinal_rf'] = y_pred_validation_rf


# Guardar las predicciones en un archivo
validation_with_predictions = DIR_DATA + 'Casos_de_Dengue_validation_with_predictions.csv'
data_validation.to_csv(validation_with_predictions, index=False)

print(f"Predicciones guardadas en: {validation_with_predictions}")

In [ ]:
import matplotlib.pyplot as plt

# Obtener los valores y las proporciones
values = data_validation['pred_clasfinal_rf'].value_counts(normalize=True)

# Crear el gráfico de barras
plt.figure(figsize=(8, 5))
values.plot(kind='bar', color='skyblue', alpha=0.7)

# Personalizar el gráfico
plt.title('Distribución de Predicciones (RF)', fontsize=14)
plt.xlabel('Clases Predichas', fontsize=12)
plt.ylabel('Proporción', fontsize=12)
plt.xticks(rotation=0, fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar el gráfico
plt.tight_layout()
plt.show()


In [46]:
RF_model_accuracy = accuracy_score(y_validation,y_pred_validation_rf)
RF_precision = precision_score(y_validation,y_pred_validation_rf)
RF_recall = recall_score(y_validation,y_pred_validation_rf)
RF_f1 = f1_score(y_validation,y_pred_validation_rf)

In [ ]:
import matplotlib.pyplot as plt

# Valores de las métricas (reemplazar con tus valores si no son variables)
metrics = {
    'Accuracy': RF_model_accuracy,
    'Precision': RF_precision,
    'Recall': RF_recall,
    'F1 Score': RF_f1
}

# Crear el gráfico de barras
plt.figure(figsize=(8, 5))
plt.bar(metrics.keys(), metrics.values(), color='skyblue', alpha=0.8)

# Personalizar el gráfico
plt.title('Evaluación del Modelo Random Forest', fontsize=14)
plt.xlabel('Métricas', fontsize=12)
plt.ylabel('Valores', fontsize=12)
plt.ylim(0, 1.1)  # Escala para métricas entre 0 y 1
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Añadir etiquetas de valor en las barras
for i, (metric, value) in enumerate(metrics.items()):
    plt.text(i, value + 0.02, f"{value:.2f}", ha='center', fontsize=10)

# Mostrar el gráfico
plt.tight_layout()
plt.show()


# ***Analisis de Resultados***

El dataset usado para validar el modelo, arrojo los siguientes resultados:

- 51% de los casos de Dengue presenta sintomas alarmantes; el 48% restante presenta sintomas no alarmantes.
- El modelo arrojo una exactitud del 92% y una precisión del 88%, delatando así las buenas predicciónes que esta haciendo el modelo.

Un modelo hecho para la predicción de casos alarmantes de Dengue y para ayudar a mitigar el traumatismo que puede generar dicha enfermedad. 

In [ ]:
data_validation.columns

In [49]:
symptom_cols = [
    'fiebre', 'cefalea', 'dolrretroo', 'malgias', 'artralgia', 'erupcionr', 
    'dolor_abdo', 'vomito', 'diarrea', 'hipotensio', 'hepatomeg', 'hem_mucosa', 
    'hipotermia', 'aum_hemato', 'caida_plaq', 'acum_liqui'
]

In [50]:
clasfinal_1 = data_validation[data_validation['clasfinal'] == 1]
clasfinal_0 = data_validation[data_validation['clasfinal'] == 0]

In [51]:
userAlarming = clasfinal_1.iloc[0]
userNoAlarming = clasfinal_0.iloc[0]

In [52]:
user_symptom = userAlarming[symptom_cols]

In [ ]:
user_symptom

In [ ]:
symptomsUserAlarming = user_symptom[user_symptom == 1].index.tolist()  

symptomsUserAlarming

In [ ]:
user_symptom = userNoAlarming[symptom_cols]
symptomsUserNoAlarming= user_symptom[user_symptom == 1].index.tolist()  

symptomsUserNoAlarming

In [ ]:
user_symptom

In [56]:
API_KEY = System.getenv("API_KEY")
MODEL_NAME = "claude-3-haiku-20240307"

In [57]:
client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [58]:
predict_model_userAlarming = userAlarming['pred_clasfinal_rf']

In [59]:
predict_model_UserNoAlarming = userNoAlarming['pred_clasfinal_rf']

In [60]:
symptoms = {
    'fiebre': 'fiebre',
    'cefalea': 'dolor de cabeza',
    'dolrretroo': 'dolor retroocular',
    'malgias': 'malestar general',
    'artralgia': 'dolor articular',
    'erupcionr': 'erupción en la piel',
    'dolor_abdo': 'dolor abdominal',
    'vomito': 'vómitos',
    'diarrea': 'diarrea',
    'hipotensio': 'hipotensión',
    'hepatomeg': 'hepatomegalia',
    'hem_mucosa': 'hemorragia en mucosas',
    'hipotermia': 'hipotermia',
    'aum_hemato': 'aumento de hematocrito',
    'caida_plaq': 'caída de plaquetas',
    'acum_liqui': 'acumulación de líquidos'
}

In [61]:
def dataR(predict_model) -> str:
        
    if predict_model == 1:
        return "con sintomas de alarma"
    else:
        return "sin sintomas de alarma"
        

In [62]:
symptomsUserAlarming = [symptoms[sintoma] for sintoma in symptomsUserAlarming] 

In [63]:
symptomsUserNoAlarming = [symptoms[sintoma] for sintoma in symptomsUserNoAlarming] 

In [ ]:
SYSTEM_PROMPT = (
    "Chat, tienes un paciente con dengue. Antes de que acuda al médico, "
    "proporciona una lista de recomendaciones y restricciones específicas para el paciente "
    "en función de los síntomas presentados. Asegúrate de incluir indicaciones claras sobre "
    "alimentos recomendados y alimentos que debe evitar, así como restricciones relacionadas "
    "con actividades físicas o o cualquier sintoma al que se debe tener precaucion. Si mencionas medicamentos, "
    "indica claramente que deben ser administrados bajo supervisión médica. La respuesta debe "
    "estar estructurada y únicamente contener las recomendaciones y restricciones, sin incluir "
    "justificaciones adicionales o explicaciones innecesarias."
)


PROMPT = f"El paciente tiene dengue  {dataR(predict_model_userAlarming)} y tiene lo siguientes sintomas: {symptomsUserAlarming} "
response_alarming =get_completion(PROMPT, SYSTEM_PROMPT)
print(response_alarming)

In [ ]:
SYSTEM_PROMPT = (
    "Chat, tienes un paciente con dengue. Antes de que acuda al médico, "
    "proporciona una lista de recomendaciones y restricciones específicas para el paciente "
    "en función de los síntomas presentados. Asegúrate de incluir indicaciones claras sobre "
    "alimentos recomendados y alimentos que debe evitar, así como restricciones relacionadas "
    "con actividades físicas o cualquier sintoma al que se debe tener precaucion. Si mencionas medicamentos, "
    "indica claramente que deben ser administrados bajo supervisión médica. La respuesta debe "
    "estar estructurada y únicamente contener las recomendaciones y restricciones, sin incluir "
    "justificaciones adicionales o explicaciones innecesarias."
)

PROMPT = (
    f"El paciente tiene dengue identificado mediante {dataR(predict_model_UserNoAlarming)} "
    f"y presenta los siguientes síntomas: {symptomsUserNoAlarming}. "
    "Proporciona recomendaciones y restricciones para este caso."
    )
response_no_alarming = get_completion(PROMPT, SYSTEM_PROMPT)
print(response_no_alarming)

***Función para generar el PDF***

In [124]:
def generar_pdf(nombre_archivo, contenido, paciente, edad):
    """
    Genera un archivo PDF con el contenido dado en un formato de informe formal.
    Args:
        nombre_archivo (str): Nombre del archivo PDF.
        contenido (str): Texto a incluir en el PDF.
        paciente (str): Nombre del paciente.
        edad (int): Edad del paciente.
    """
    c = canvas.Canvas(nombre_archivo, pagesize=letter)
    width, height = letter

    
    margen_x, margen_y = 50, 50
    line_height = 14
    max_chars_per_line = 90
    x, y = margen_x + 10, height - margen_y - 20

    
    c.setStrokeColor(black)
    c.setFillColor(white)
    c.rect(margen_x - 10, margen_y - 10, width - 2 * margen_x + 20, height - 2 * margen_y + 20, stroke=1, fill=1)

 
    c.setFont("Times-Bold", 18)
    c.setFillColor(black)
    c.drawString(margen_x, height - margen_y - 40, "Sistema de predicciones de casos de dengue")
    c.drawImage("mosquito.png", width - margen_x - 80, height - margen_y - 70, width=80, height=50, mask='auto')

    
    c.setFont("Times-Roman", 12)
    fecha_actual = date.today().strftime("%d/%m/%Y")
    c.drawString(margen_x, height - margen_y - 90, f"Paciente: {paciente}")
    c.drawString(margen_x, height - margen_y - 110, f"Edad: {edad} años")
    c.drawString(margen_x, height - margen_y - 130, f"Fecha: {fecha_actual}")

    
    c.line(margen_x + 10, height - margen_y - 140, width - margen_x - 35, height - margen_y - 140)

    
    y = height - margen_y - 160
    for linea in contenido.split("\n"):
        while len(linea) > max_chars_per_line:
            fragmento = linea[:max_chars_per_line]
            c.drawString(x, y, fragmento)
            linea = linea[max_chars_per_line:]
            y -= line_height

            
            if y < margen_y + 30:
                c.showPage()
                c.setStrokeColor(black)
                c.setFillColor(white)
                c.rect(margen_x - 10, margen_y - 10, width - 2 * margen_x + 20, height - 2 * margen_y + 20, stroke=1, fill=1)
                c.setFont("Times-Roman", 12)
                y = height - margen_y - 20

        c.drawString(x, y, linea)
        y -= line_height

        
        if y < margen_y + 30:
            c.showPage()
            c.setStrokeColor(black)
            c.setFillColor(white)
            c.rect(margen_x - 10, margen_y - 10, width - 2 * margen_x + 20, height - 2 * margen_y + 20, stroke=1, fill=1)
            c.setFont("Times-Roman", 12)
            y = height - margen_y - 20

    
    y -= line_height * 2 
    c.setFont("Times-Italic", 10)
    c.setFillColor(black)
    footer_text = (
        "Estas recomendaciones son provisionales y aplican en caso de no tener acceso inmediato a un médico. "
        "Se recomienda encarecidamente acudir a un profesional de la salud para una evaluación adecuada."
    )
    
    footer_lines = []
    while footer_text:
        if len(footer_text) > max_chars_per_line:
            split_index = footer_text.rfind(' ', 0, max_chars_per_line)
            if split_index == -1:
                split_index = max_chars_per_line
            footer_lines.append(footer_text[:split_index])
            footer_text = footer_text[split_index:].strip()
        else:
            footer_lines.append(footer_text)
            footer_text = ""

    for footer_line in footer_lines:
        c.drawString(margen_x, y, footer_line)
        y -= line_height

  
    c.save()


In [ ]:
nombre_pdf_alarming = "Paciente_Alarmante.pdf"
generar_pdf(nombre_pdf_alarming, f"{response_alarming}","paciente",userAlarming["edad_"])
print(f"PDF generado: {nombre_pdf_alarming}")

In [ ]:
nombre_pdf_no_alarming = "Paciente_No_Alarmante.pdf"
generar_pdf(nombre_pdf_no_alarming, f"{response_no_alarming}","paciente",userNoAlarming["edad_"])
print(f"PDF generado: {nombre_pdf_no_alarming}")